## Generating a data model for CLIWOC

### Creating a data model

In [1]:
from __future__ import annotations

import json
import os
import shutil

import pandas as pd

from cdm_reader_mapper import mdf_reader, test_data

try:
    from importlib.resources import files as get_files
except ImportError:
    from importlib_resources import files as get_files

from tempfile import TemporaryDirectory

2024-06-06 10:48:27,524 - root - INFO - init basic configure of logging success
/Users/josidd/Projects/cdm_glamod/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-06 10:48:30,630 - root - INFO - init basic configure of logging success


In [2]:
schema = "imma1"
data_file_path = test_data.test_063_714["source"]
data_raw = mdf_reader.read(data_file_path, data_model=schema)

2024-06-06 10:48:30,755 - root - INFO - Attempting to fetch remote file: imma1_714/input/063-714_2010-07-01_subset.imma.md5
2024-06-06 10:48:31,193 - root - INFO - READING DATA MODEL SCHEMA FILE...
2024-06-06 10:48:31,199 - root - INFO - EXTRACTING DATA FROM MODEL: imma1
2024-06-06 10:48:31,200 - root - INFO - Getting data string from source...
2024-06-06 10:48:31,337 - root - WARNING - Data numeric elements with missing upper or lower threshold: ('c1', 'BSI'),('c1', 'AQZ'),('c1', 'AQA'),('c1', 'UQZ'),('c1', 'UQA'),('c1', 'VQZ'),('c1', 'VQA'),('c1', 'PQZ'),('c1', 'PQA'),('c1', 'DQZ'),('c1', 'DQA'),('c5', 'OS'),('c5', 'OP'),('c5', 'FM'),('c5', 'IMMV'),('c5', 'IX'),('c5', 'W2'),('c5', 'WMI'),('c5', 'SD2'),('c5', 'SP2'),('c5', 'IS'),('c5', 'RS'),('c5', 'IC1'),('c5', 'IC2'),('c5', 'IC3'),('c5', 'IC4'),('c5', 'IC5'),('c5', 'IR'),('c5', 'RRR'),('c5', 'TR'),('c5', 'NU'),('c5', 'QCI'),('c5', 'QI1'),('c5', 'QI2'),('c5', 'QI3'),('c5', 'QI4'),('c5', 'QI5'),('c5', 'QI6'),('c5', 'QI7'),('c5', 'QI8'

In [3]:
data_raw.data["c99"].head()

0    99 0 48683,20100701,0000,  88.3790,  43.2100,1...
1    99 0 25629,20100701,0000,  87.8090, -54.8550,1...
2    99 0 25595,20100701,0000,  87.5610,  41.7610,1...
3    99 0 26558,20100701,0000,  87.4040,  17.9720,1...
4    99 0 26559,20100701,0000,  87.4030,  17.9730,1...
Name: c99, dtype: object

In [4]:
line = data_raw.data["c99"].iloc[3]
line

'99 0 26558,20100701,0000,  87.4040,  17.9720,1,        ,    ,       ,       , ,   1.60,1,1013.60,1,       , ,       , ,       ,'

### Custom Schema

To use a custom schema we need to use the `data_model_path` argument in `mdf_reader.read`. The structure of the directory is:

```
name_of_model/
    name_of_model.json
    code_tables/
        ...
```

The `code_tables` sub-directory contains the code tables that map the key columns in the data to their values.

In this demonstration we will create a new model in a temporary directory. It will be a copy of the `imma1_d703` schema and code tables.

In [5]:
tmp_dir = TemporaryDirectory()
model_name = "imma1_d730"
my_model_path = os.path.join(tmp_dir.name, model_name)
os.mkdir(my_model_path)

# Load schema and save to json file
schema = mdf_reader.schemas.read_schema(model_name)
json_object = json.dumps(schema, indent=2)

with open(os.path.join(my_model_path, model_name + ".json"), "w") as outfile:
    outfile.write(json_object)

# Get code tables and copy to the directory
code_tables_path = get_files(
    ".".join([mdf_reader.properties._base, "code_tables", "imma1"])
)
shutil.copytree(code_tables_path, os.path.join(my_model_path, "code_tables"))

'/var/folders/vf/pskk3w4j38l8kk7bc9xm07j00000gp/T/tmpude1dlw9/imma1_d730/code_tables'

In [6]:
data_file_path = test_data.test_133_730["source"]
data = mdf_reader.read(data_file_path, data_model_path=my_model_path)

2024-06-06 10:48:31,813 - root - INFO - Attempting to fetch remote file: imma1_730/input/133-730_1776-10-01_subset.imma.md5
2024-06-06 10:48:31,930 - root - INFO - READING DATA MODEL SCHEMA FILE...
2024-06-06 10:48:31,942 - root - INFO - EXTRACTING DATA FROM MODEL: /var/folders/vf/pskk3w4j38l8kk7bc9xm07j00000gp/T/tmpude1dlw9/imma1_d730
2024-06-06 10:48:31,944 - root - INFO - Getting data string from source...
2024-06-06 10:48:32,177 - root - WARNING - Data numeric elements with missing upper or lower threshold: ('c1', 'BSI'),('c1', 'AQZ'),('c1', 'AQA'),('c1', 'UQZ'),('c1', 'UQA'),('c1', 'VQZ'),('c1', 'VQA'),('c1', 'PQZ'),('c1', 'PQA'),('c1', 'DQZ'),('c1', 'DQA'),('c5', 'OS'),('c5', 'OP'),('c5', 'FM'),('c5', 'IMMV'),('c5', 'IX'),('c5', 'W2'),('c5', 'WMI'),('c5', 'SD2'),('c5', 'SP2'),('c5', 'IS'),('c5', 'RS'),('c5', 'IC1'),('c5', 'IC2'),('c5', 'IC3'),('c5', 'IC4'),('c5', 'IC5'),('c5', 'IR'),('c5', 'RRR'),('c5', 'TR'),('c5', 'NU'),('c5', 'QCI'),('c5', 'QI1'),('c5', 'QI2'),('c5', 'QI3'),('

In [7]:
data.data[["c99_sentinal"]].head()

c99_sentinal          
          ATTI ATTL  BLK
0           99    0  NaN
1           99    0  NaN
2           99    0  NaN
3           99    0  NaN
4           99    0  NaN

In [8]:
data.data[["c99_logbook"]].c99_logbook.describe(include="all")

,InstAbbr,InsName,InsPlace,InsLand,No_data_entry,NameArchiveSet,ArchivePart,Specification,Logbook_id,Logbook_language,Image_No,Illustr
count,5,5,5,5,1,0,1,1,5,5,1,5
unique,4,4,4,4,1,0,1,1,5,4,1,1
top,AGI,ARCHIVO GENERAL DE INDIAS,SEVILLE,SPAIN,20,NaN,MCC,1391,"CORREOS, 275A R11",SPANISH,MCC_20_1391_0032,0
freq,2,2,2,2,1,NaN,1,1,1,2,1,5


In [9]:
pd.options.display.max_columns = None
data.data[["c99_voyage"]].c99_voyage.describe(include="all")

,drLatDeg,drLatMin,drLatSec,drLatHem,drLonDeg,drLonMin,drLonSec,drLonHem,LatDeg,LatMin,LatSec,LatHem,LonDeg,LonMin,LonSec,LonHem,LatInd,LonInd,ZeroMeridian,LMname1,LMdirection1,LMdistance1,LMname2,LMdirection2,LMdistance2,LMname3,LMdirection3,LMdistance4,PosCoastal,Calendar_type,logbook_date,TimeOB,Day_of_the_week,PartDay,Watch,Glasses,Start_day,ShipName,Nationality,Ship_type,Company,Name1,Rank1,Name2,Rank2,Name3,Rank3,voyage_from,voyage_to,Anchored_ind,AnchorPlace,DASno,VoyageIni,Course_ship,Ship_speed,Distance,EncName,EncNat
count,4.000000,4.000000,4.0,4,2.000000,2.00000,2.0,2,2.000000,2.000000,2.0,2,3.000000,3.000000,3.0,3,5,5,5,1,1,1.0,0,0,0.0,0,0,0.0,5,5,5,5.0,1,1,1,1.0,5,5,5,4,2,4,4,1,1,0,0,5,5,5,0,0,5,2,0,4,0,0
unique,NaN,NaN,NaN,2,NaN,NaN,NaN,1,NaN,NaN,NaN,2,NaN,NaN,NaN,2,2,3,4,1,1,NaN,0,0,NaN,0,0,NaN,1,1,1,NaN,1,1,1,<NA>,2,5,4,4,2,4,3,1,1,0,0,5,4,1,0,0,5,2,0,4,0,0
top,NaN,NaN,NaN,N,NaN,NaN,NaN,E,NaN,NaN,NaN,N,NaN,NaN,NaN,E,1,2,TENERIFE,LIZARD,N87:17E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2,17711001,NaN,TUESDAY,3,VM,<NA>,UNKNOWN,EL COLON,SPANISH,PAQUEBOTE,MCC,THOMAS D'ORVES,CAPITAN,CHARLES WARREN,2ND OFFICER/LIEUTENANT,NaN,NaN,LA HABANA,LA CORUÑA,0,NaN,NaN,17710819,WTZ,NaN,175.00,NaN,NaN
freq,NaN,NaN,NaN,3,NaN,NaN,NaN,2,NaN,NaN,NaN,1,NaN,NaN,NaN,2,3,2,2,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,5,5,NaN,1,1,1,<NA>,4,1,2,1,1,1,2,1,1,NaN,NaN,1,2,5,NaN,NaN,1,1,NaN,1,NaN,NaN
mean,27.250000,24.250000,0.0,NaN,26.500000,36.00000,0.0,NaN,22.000000,9.500000,0.0,NaN,121.666667,42.666667,0.0,NaN,NaN,NaN,NaN,NaN,NaN,230.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,21.884165,16.879475,0.0,NaN,19.091883,19.79899,0.0,NaN,29.698485,13.435029,0.0,NaN,195.208436,11.239810,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,5.000000,0.0,NaN,13.000000,22.00000,0.0,NaN,1.000000,0.000000,0.0,NaN,4.000000,33.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,230.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,13.750000,17.000000,0.0,NaN,19.750000,29.00000,0.0,NaN,11.500000,4.750000,0.0,NaN,9.000000,36.500000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,230.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,29.500000,23.000000,0.0,NaN,26.500000,36.00000,0.0,NaN,22.000000,9.500000,0.0,NaN,14.000000,40.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,230.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,43.000000,30.250000,0.0,NaN,33.250000,43.00000,0.0,NaN,32.500000,14.250000,0.0,NaN,180.500000,47.500000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,230.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
data.data[["c99_voyage"]].c99_voyage.ZeroMeridian.head()

0     TENERIFE
1    GREENWICH
2      NL_0_01
3      BERMUDA
4     TENERIFE
Name: ZeroMeridian, dtype: object

e.g. the ship types on this deck will be given in a tons of different languages. There is no code table for this variable in the CLIWOC website.

In [11]:
data.data[["c99_voyage"]].c99_voyage.Ship_type.dropna().head()

0    PAQUEBOTE
2        SNAUW
3     5TH RATE
4     PAQUEBOT
Name: Ship_type, dtype: object

In [12]:
data.data[["c99_data"]].c99_data.describe(include="all")

,AT_reading_units,SST_reading_units,AP_reading_units,BART_reading_units,ReferenceCourse,ReferenceWindDirection,Decl,Distance_units,Distance_units_to_landmark,Distance_units_travelled,Longitude_units,units_of_measurement,humidity_units,water_at_pump_units,wind_scale,BARO_type,BARO_brand,API,Humidity_method,compas_error,compas_correction,AT_outside,SST,AP,wind_dir,current_dir,current_speed,attached_tem,pump_water,Humidity,wind_force,weather,prcp_descriptor,sea_state,shape_coulds,dir_coulds,Clearness,cloud_fraction,gusts,Rain,Fog,Snow,Thunder,Hail,Sea_ice,Trivial_correction,Release
count,0,0,0,0,2,5,5,0,1,4,5,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,5,0,0,0.0,0,0,5,2,0,4,0,0,0,0,5,5,5,5,5,5,5,5,5
unique,0,0,0,0,1,1,5,0,1,4,2,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,0,0,NaN,0,0,5,2,0,4,0,0,0,0,1,2,1,1,2,1,1,1,2
top,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN,-20,NaN,LEAGUES,MILLAS,360 DEGREES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN,EN REFREGONES FUERTES Y DESPUES BONANCIBLE,"MUY MALOS CARICES. AGUACEROS, RELAMPAGOS Y TRU...",NaN,GRANDE DEL O Y DEL ENE,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,CLIWOC VERSION 2.0
freq,NaN,NaN,NaN,NaN,2,5,1,NaN,1,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1,1,NaN,1,NaN,NaN,NaN,NaN,5,4,5,5,4,5,5,5,4
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


What about the different scales for the wind force, given different languages?

In [13]:
data.data[["c99_data"]].c99_data.wind_force.head()

0    EN REFREGONES FUERTES Y DESPUES BONANCIBLE
1                                        FOIBLE
2               STIJVE GEREEFDE MARSZEILSKOELTE
3                       FRESH GALES AND SQUALLY
4                                    BONANCIBLE
Name: wind_force, dtype: object